In [1]:
!wget https://raw.githubusercontent.com/IvanShcherbak/psychologyst/main/data/train_ext.csv
!wget https://raw.githubusercontent.com/IvanShcherbak/psychologyst/main/data/test_ext.csv
!wget https://raw.githubusercontent.com/IvanShcherbak/psychologyst/main/data/valid_ext.csv
!wget https://raw.githubusercontent.com/IvanShcherbak/psychologyst/main/source/bert_dataset.py
!wget https://raw.githubusercontent.com/IvanShcherbak/psychologyst/main/source/bert_classifier.py

--2023-12-07 13:28:01--  https://raw.githubusercontent.com/IvanShcherbak/psychologyst/main/data/train_ext.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9702245 (9.3M) [text/plain]
Saving to: ‘train_ext.csv’

train_ext.csv       100%[===================>]   9.25M  --.-KB/s    in 0.1s    

2023-12-07 13:28:01 (72.3 MB/s) - ‘train_ext.csv’ saved [9702245/9702245]

--2023-12-07 13:28:01--  https://raw.githubusercontent.com/IvanShcherbak/psychologyst/main/data/test_ext.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3056623 (

In [2]:
import pandas as pd

In [3]:
train_data = pd.read_csv('train_ext.csv')
valid_data = pd.read_csv('valid_ext.csv')
test_data  = pd.read_csv('test_ext.csv')

In [4]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

In [5]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=10,
        model_save_path='bert_ext.pt'
)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

In [6]:
classifier.preparation(
        X_train=list(train_data['TEXT']),
        y_train=list(train_data['cEXT']),
        X_valid=list(valid_data['TEXT']),
        y_valid=list(valid_data['cEXT'])
    )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
classifier.train()

Epoch 1/10
Train loss 0.7019421820102687 accuracy 0.5031685678073511
Val loss 0.6933385708717384 accuracy 0.4860759493670886
----------
Epoch 2/10
Train loss 0.683961785187135 accuracy 0.5538656527249683
Val loss 0.6961316576208731 accuracy 0.5164556962025316
----------
Epoch 3/10
Train loss 0.6502784391621975 accuracy 0.632446134347275
Val loss 0.7633443020208918 accuracy 0.5544303797468354
----------
Epoch 4/10
Train loss 0.6364696543328511 accuracy 0.7008871989860583
Val loss 1.1861616352836442 accuracy 0.5569620253164557
----------
Epoch 5/10
Train loss 0.731428065575088 accuracy 0.7389100126742713
Val loss 1.4847053070176355 accuracy 0.5645569620253165
----------
Epoch 6/10
Train loss 0.686797172573002 accuracy 0.7870722433460076
Val loss 1.9125524441561088 accuracy 0.5544303797468354
----------
Epoch 7/10
Train loss 0.6280132760286404 accuracy 0.829531051964512
Val loss 2.189368904701721 accuracy 0.5240506329113924
----------
Epoch 8/10
Train loss 0.5995425627795868 accuracy 0.84

In [8]:
texts = list(test_data['TEXT'])
labels = list(test_data['cEXT'])

predictions = [classifier.predict(t) for t in texts]

In [9]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.5420118343195266, recall: 0.5380719035126796, f1score: 0.5321969696969697
